# Lab 3 — clustering

In [ ]:
# Given Imports
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle

# Import bokeh
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool, ResetTool, PanTool, WheelZoomTool, SaveTool
output_notebook()
# Categorial colors
from bokeh.palettes import Dark2_8

%matplotlib inline
plt.style.use("ggplot")

### Exercise 3.11 : Clustering tags

In [ ]:
# Let's open the pickle to load it.
with open( "tagCoord.pickle", "rb" ) as f:
    tagEmbed = pk.load( f, encoding="utf-8" )

In [ ]:
#Cluster the data using the k-means clustering algorithm.

# Input: N data points {x1, ..., xn} , given in our pickle.
#        K number of clusters, between 2 and 5 for us.
# Output: We will have mu_k, center of clusters, K times.
#         r_nk, point-cluster assigment indicator. If r_nk = 1, n is in cluster k.

# r_nk = 1 only for k = argmin ||xn - mu_k||
# mu_k = sum_over_n(r_nk + xn)/sum_over_n(r_nk)

# Start with K = 2


In [ ]:
# Use the categorial colors for clustering.
source = ColumnDataSource(
    data={
        "x": [x[0] for x in data],
        "y": [x[1] for x in data],
        "name": [x[2] for x in data],
        "color": [Dark2_8[x[4]] for x in data],
    })

hover = HoverTool(
    tooltips=[
        ("Name", "@name"),
    ])
tools = [hover, ResetTool(), PanTool(), WheelZoomTool(), SaveTool()]

p = figure(plot_width=960, plot_height=360, tools=tools, title="Mouse over the dots")
p.circle("x", "y", source=source, size=20, color="color", alpha=0.5)
show(p, notebook_handle=True)

### Exercise 3.12 : Clustering movies